In [1]:
# !pip install pytube

In [2]:
# importing DS module
import json
import glob
import joblib
import numpy as np
import pandas as pd

# importing data processing module  
import cv2
from pytube import YouTube  


from tqdm.notebook import tqdm

In [3]:
# read path
DATA_DIR = '../data'

# where to save  
RAW_VIDEO_PATH = "../data/raw/video/train"
PREPROCESS_VIDEO_PATH = "../data/preprocessed/video/train"

In [4]:
# read the json file
with open(DATA_DIR + '/metadata/MS-ASL/MSASL_train.json') as f:
  train_data = json.load(f)

# turn json to np array
train_data = pd.DataFrame(train_data)
train_data.head()

,org_text,clean_text,start_time,signer_id,signer,start,end,file,label,height,fps,end_time,url,text,box,width,review
0,match [light-a-MATCH],match,0.0,0,0,0,83,match light-a-MATCH,830,360.0,30.00,2.767,https://www.youtube.com/watch?v=C37R_Ix8-qs,match,"[0.05754461884498596, 0.21637457609176636, 1.0...",640.0,NaN
1,FAIL,fail,0.0,0,-1,0,74,FAIL,542,360.0,25.00,2.960,https://www.youtube.com/watch?v=PIsUJl8BN_I,fail,"[0.0657794177532196, 0.16717177629470825, 0.93...",480.0,NaN
2,laugh,laugh,0.0,4,26,0,31,SignSchool Laugh with Legs 2,312,360.0,29.97,1.034,www.youtube.com/watch?v=9FdHlMOnVjg,laugh,"[0.13188594579696655, 0.32334136962890625, 1.0...",640.0,NaN
3,BOOK,book,0.0,0,-1,0,66,BOOK(3),38,360.0,25.00,2.640,https://www.youtube.com/watch?v=J7tP98oDxqE,book,"[0.05569887161254883, 0.25173279643058777, 0.9...",480.0,NaN
4,sign-language,sign language,0.0,0,-1,0,75,SIGN-LANGUAGE-S-CLAW-F,848,360.0,29.97,2.502,www.youtube.com/watch?v=N2mG9ZKjrGA,sign language,"[0.03904399275779724, 0.24198183417320251, 1.0...",640.0,NaN


# Convert Video to Numpy
Download the video and convert cette video to numpy array with video id, video array, and clean text label

In [5]:
for i, row in tqdm(train_data.iterrows()):
    # wait 20 seconds
    # time.sleep(20)

    # link of the video to be downloaded  
    link = row['url']
    id = link.split('=')[1]

    # Try downloading the video
    try:  
        # object creation using YouTube 
        # which was imported in the beginning  
        yt = YouTube(link)  
        # downloading the video  
        yt.streams.filter(progressive=True, file_extension='mp4').order_by('resolution')[-1].download(RAW_VIDEO_PATH, filename=id+".mp4") 

    except Exception as e:  
        print(f"Error: {e}")  




0it [00:00, ?it/s]

Error: 9FdHlMOnVjg is a private video
Error: 1AyT77LqJzQ is unavailable
Error: 1AyT77LqJzQ is unavailable
Error: cJOyCgIKyeA is unavailable
Error: zqfcp6NRKpA is a private video
Error: 7y5Ye-2-ZBs is a private video
Error: rYW7VFPi9_w is a private video
Error: 0Beq_NIDj2c is unavailable
Error: 0Beq_NIDj2c is unavailable
Error: AoQAPgEUIAs is unavailable
Error: 25BrmaKbzYo is a private video
Error: CZkLkFnsNBA is unavailable
Error: U_nbv5Mq00c is unavailable
Error: trqyl20lEno is unavailable
Error: oySXUMX7nRY is a private video
Error: IncompleteRead(0 bytes read, 189958 more expected)
Error: 3DbWOEtUigU is unavailable
Error: o_Qs0qUiJN8 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private video
Error: gI_6r7VCzM4 is a private 

# Experimental Code

In [6]:
video_arrays = []

for i, row in tqdm(train_data.iterrows()):
    # wait 20 seconds
    # time.sleep(20)

    # link of the video to be downloaded  
    link = row['url']
    id = link.split('=')[1]
    label = row['clean_text']


    # Initialize a VideoCapture object to read from the video file
    video_path = RAW_VIDEO_PATH+f'/{id}.mp4'
    cap = cv2.VideoCapture(video_path)


    # Check if the video was opened successfully
    if not cap.isOpened():
        print("Error: Could not open video.")
        continue


    # Read the frames from the video
    frames = []
    while True:
        ret, frame = cap.read()
        if not ret:
            break  # Break the loop if there are no frames left to read

        # Append the frame to the list of frames
        frames.append(frame)

    # Convert the list of frames to a NumPy array
    video_array = np.array(frames)

    # Release the VideoCapture object
    cap.release()

    # append the video array to the list of video arrays
    data = (id, video_array, label)
    video_arrays.append(data)


# save data as pickle file
joblib.dump(video_arrays, f'{PREPROCESS_VIDEO_PATH}/train.pkl')

0it [00:00, ?it/s]

Error: Could not open video.
Error: Could not open video.
Error: Could not open video.
Error: Could not open video.
Error: Could not open video.
Error: Could not open video.
Error: Could not open video.


: 